In [1]:
import json
from azure.eventhub import EventHubProducerClient, EventData
from dotenv import load_dotenv
import os
import socket
import random

from random import randrange

load_dotenv()
EVENT_HUB_CONN_STRING = os.getenv("EVENT_HUB_CONN_STRING") 
EVENT_HUB_NAME_IMPRESSIONS = os.getenv("EVENT_HUB_NAME_IMPRESSIONS")
EVENT_HUB_NAME_CLICKS = os.getenv("EVENT_HUB_NAME_CLICKS")

eventHubConnString = EVENT_HUB_CONN_STRING
eventHubNameImpressions = EVENT_HUB_NAME_IMPRESSIONS
eventHubNameClicks = EVENT_HUB_NAME_CLICKS

producer_impressions = EventHubProducerClient.from_connection_string(conn_str=eventHubConnString, eventhub_name=eventHubNameImpressions)
producer_clicks = EventHubProducerClient.from_connection_string(conn_str=eventHubConnString, eventhub_name=eventHubNameClicks)

hostname = socket.gethostname()

In [2]:
from utils import generateClickEvent, generateImpressionEvent

def sendToEventsHub(jsonEvent, producer):
    eventString = json.dumps(jsonEvent)
    print(eventString) 
    event_data_batch = producer.create_batch() 
    event_data_batch.add(EventData(eventString)) 
    producer.send_batch(event_data_batch)

def generateEvents(isUnsupportedBrowser = False, isAnomaly = False):
    try:
        while True:
            impressionEvent = generateImpressionEvent(isUnsupportedBrowser, isAnomaly, None)    
            sendToEventsHub(impressionEvent, producer_impressions)
            if random.randint(1, 100) > 80:
                clickEvent = generateClickEvent(impressionEvent)    
                sendToEventsHub(clickEvent, producer_clicks)
    except KeyboardInterrupt:
        producer_impressions.close()
        producer_clicks.close()


In [3]:
generateEvents(False, False)

{"impressionId": "dac3fd96-96da-4c46-bf18-0d572e70969a", "impressionDate": "2024-01-05T10:13:09", "productId": 860, "browser": "Edge", "browserVersion": "14.6", "device": "computer", "source": "google", "ip_address": "143.51.117.174", "landing_page": "https://www.pittman.com/appregister.asp", "page_loading_seconds": 9.839}
{"impressionId": "ab5ce0c6-e8f0-48d8-9a85-5c369118bfc2", "impressionDate": "2024-01-08T03:31:19", "productId": 747, "browser": "Safari", "browserVersion": "6.6", "device": "mobile", "source": "bing", "ip_address": "69.172.134.137", "landing_page": "http://hahn.info/tag/search/blogpost.php", "page_loading_seconds": 9.654}
{"impressionId": "d3cff2e4-245a-4b51-8ab8-cd38dbe4d093", "impressionDate": "2024-01-05T03:13:22", "productId": 899, "browser": "Chrome", "browserVersion": "13.6", "device": "mobile", "source": "bing", "ip_address": "47.110.249.34", "landing_page": "http://www.baker.com/app/appsearch.html", "page_loading_seconds": 1.511}
{"impressionId": "a09c759c-2aa